Scrape missing data from doesthedogdie.com

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sys
sys.path.append('../utils')
import functions

In [ ]:
df = pd.read_csv('../data/local/clean/films_19to24.csv')
display(df)